In [ ]:
from tensorflow.keras import models, layers, regularizers
import numpy as np
import pandas as pd

In [ ]:
#Data Load
excel = pd.read_csv('/content/drive/MyDrive/anndl_csv/breast-cancer-wisconsin_copy.data.csv')
excel

In [ ]:
#Benign, Malignant를 0과 1로 변경하고 필요없는 ID number column 제거
breast_cancer = pd.DataFrame(excel)
breast_cancer['Class'] = breast_cancer['Class'].replace(2,0)
breast_cancer['Class'] = breast_cancer['Class'].replace(4,1)
breast_cancer = breast_cancer.drop('Sample code number',axis=1)
#Prepare한 데이터 확인
breast_cancer

Data Preprocessing

In [ ]:
data = breast_cancer.drop('Class',axis=1).to_numpy()
target = breast_cancer['Class'].to_numpy()
#Train data, Test data split
test_data = data[:100]
test_targets = target[:100]
train_data = data[100:]
train_targets = target[100:]
#Normalization
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

#Validation(Train Data에서 Validation으로 사용할 데이터 split)
val_data = train_data[0:100]
val_targets = train_targets[0:100]
partial_train_data = train_data[100:]
partial_train_targets = train_targets[100:]

Modeling

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(10,activation='relu',input_shape=(9,)))
  model.add(layers.Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
  return model

Training & Evaluation with repeating #

In [ ]:
repeat = 5
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for i in range(repeat):
  from tensorflow.keras.callbacks import EarlyStopping
  num_epochs = 200
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=10, 
                    callbacks=[EarlyStopping(monitor='val_loss',patience=2)],verbose=0)
  train_loss.append(history.history['loss'][-1]) #Train에서 얻은 마지막 loss값 얻기
  train_acc.append(history.history['accuracy'][-1]) #Train에서 얻은 마지막 accuracy값 얻기
  
  eval_loss, eval_acc = model.evaluate(test_data,test_targets)
  test_loss.append(eval_loss)
  test_acc.append(eval_acc)


In [ ]:
print('train_loss',train_loss) 
print('train_acc',train_acc) 
print('test_loss:',test_loss)
print('test_acc:',test_acc)

In [ ]:
# Mean & Std 구하기
train_loss_mean = np.mean(train_loss)
train_loss_std = np.std(train_loss)
print('<Train Loss>')
print('mean:',train_loss_mean,'std:',train_loss_std)

train_acc_mean = np.mean(train_acc)
train_acc_std = np.std(train_acc)
print('<Train Accuracy>')
print('mean:',train_acc_mean,'std:',train_acc_std)

test_loss_mean = np.mean(test_loss)
test_loss_std = np.std(test_loss)
print('<Test Loss>')
print('mean:',test_loss_mean,'std:',test_loss_std)

test_acc_mean = np.mean(test_acc)
test_acc_std = np.std(test_acc)
print('<Test Accuracy>')
print('mean:',test_acc_mean,'std:',test_acc_std)

Weight & Bias 값 출력

In [ ]:
w = model.get_weights()[0]
b = model.get_weights()[1]
print('Weight:',w,'\nBias:',b)

Prediction

In [ ]:
model.predict(test_data)